In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import math
from google.colab import drive
import statsmodels.api as sm
import pickle



In [ ]:

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the base path and data path
base_path = '/content/drive/Shared drives/Deloitte'
data_path = base_path + '/Original Data'
#Path for the model
base_path = '/content/drive/Shared drives/Deloitte'
model_path = base_path + '/Model'

customer_potential = pd.read_excel(data_path + '/customer_potential.xlsx')

customer_potential.head(5)



Mounted at /content/drive


,Rating,Number.of.Reviews,Distance,City,Store.Size,Opening.Hours,Phone.Number,Number.of.Employees,Free.Wifi,Parking,Description,Zip.Code,Latitude,Longitude,Date.Opening,ID
0,3.184,3,38.506364,Madrid,220,11:00 AM - 11:00 PM,938887612,31,0,1,"Whimsical ambiance, soft lighting, and aromati...",28000,40.366157,-3.675629,2029-09-16,19914
1,2.454,10,32.895455,Alicante,405,11:00 AM - 11:00 PM,918230016,31,1,0,Vibrant eatery serving a diverse array of shar...,3000,38.362899,-0.512787,2029-09-08,10537
2,2.492,7,36.050057,Barcelona,4603,9:00 AM - 9:00 PM,946142438,33,1,0,"Sleek, boutique-inspired haven nestled in the ...",8000,41.469548,2.182110,2028-12-20,17477
3,2.750,4,39.215338,Alicante,668,11:00 AM - 11:00 PM,938399524,23,0,1,Savory aromas waft from the kitchen of Bella V...,3000,38.365815,-0.526732,2028-11-16,25714
4,2.487,12,35.995439,Madrid,864,11:00 AM - 11:00 PM,950017962,30,1,1,Vibrant Spanish cuisine served with warmth and...,28000,40.402235,-3.709612,2028-07-04,23921


In [ ]:
#checking the shape of the dataset
customer_potential.shape

(38571, 16)

In [ ]:
#checking for duplicates
customer_potential.duplicated().sum()

0

## Column Names

In [ ]:
#Checking for right column names
print(customer_potential.columns)

Index(['Rating', 'Number.of.Reviews', 'Distance', 'City', 'Store.Size',
       'Opening.Hours', 'Phone.Number', 'Number.of.Employees', 'Free.Wifi',
       'Parking', 'Description', 'Zip.Code', 'Latitude', 'Longitude',
       'Date.Opening', 'ID'],
      dtype='object')


In [ ]:
#Rename column names
customer_potential.columns = customer_potential.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('.', '_')
print(customer_potential.columns)

Index(['rating', 'number_of_reviews', 'distance', 'city', 'store_size',
       'opening_hours', 'phone_number', 'number_of_employees', 'free_wifi',
       'parking', 'description', 'zip_code', 'latitude', 'longitude',
       'date_opening', 'id'],
      dtype='object')


## Column types

In [ ]:
#checking for data types
customer_potential.dtypes

rating                        float64
number_of_reviews               int64
distance                      float64
city                           object
store_size                      int64
opening_hours                  object
phone_number                    int64
number_of_employees             int64
free_wifi                       int64
parking                         int64
description                    object
zip_code                        int64
latitude                      float64
longitude                     float64
date_opening           datetime64[ns]
id                              int64
dtype: object

## Null Values

In [ ]:
#checking for null values
customer_potential.isna().sum()

rating                   0
number_of_reviews        0
distance                 0
city                   181
store_size               0
opening_hours            0
phone_number             0
number_of_employees      0
free_wifi                0
parking                  0
description              0
zip_code                 0
latitude                 0
longitude                0
date_opening             0
id                       0
dtype: int64

## Rename City

In [ ]:
customer_potential['city'].unique()
#not consistent

array([' Madrid', 'Alicante', 'Barcelona', 'Bilbao', 'Sevilla',
       'A Coruna', 'Valencia', 'Zaragoza', nan, 'A  CoruÃ±a', 'Caceres',
       'Pamplona', 'Valladolid', 'Burgos', 'Guadalajara', 'Balencia'],
      dtype=object)

In [ ]:
customer_potential['city'] = customer_potential['city'].str.strip()

In [ ]:
#Checking the zip code belonging to the cities
printed_cities = set()

for index, row in customer_potential.iterrows():
    if row['city'] not in printed_cities:
        print(f"City: {row['city']}, Zip Code: {row['zip_code']}")
        printed_cities.add(row['city'])

City: Madrid, Zip Code: 28000
City: Alicante, Zip Code: 3000
City: Barcelona, Zip Code: 8000
City: Bilbao, Zip Code: 48000
City: Sevilla, Zip Code: 41000
City: A Coruna, Zip Code: 15000
City: Valencia, Zip Code: 46000
City: Zaragoza, Zip Code: 50000
City: nan, Zip Code: 30000
City: A  CoruÃ±a, Zip Code: 15000
City: Caceres, Zip Code: 10000
City: Pamplona, Zip Code: 31000
City: Valladolid, Zip Code: 47000
City: Burgos, Zip Code: 9000
City: Guadalajara, Zip Code: 19000
City: Balencia, Zip Code: 34000


In [ ]:
#Align all the names
city_mapping = {
    'A  CoruÃ±a': 'A Coruña',
    'A Coruna': 'A Coruña',
    'Balencia': 'Palencia',
}

customer_potential['city'] = customer_potential['city'].replace(city_mapping)

print(customer_potential['city'].unique())

['Madrid' 'Alicante' 'Barcelona' 'Bilbao' 'Sevilla' 'A Coruña' 'Valencia'
 'Zaragoza' nan 'Caceres' 'Pamplona' 'Valladolid' 'Burgos' 'Guadalajara'
 'Palencia']


## Fillling in NA: Zip Codes of the NA's to find the correct city that belongs to this zip code


In [ ]:
printed_zip_codes = set()

for zip_code in customer_potential.loc[customer_potential['city'].isna(), 'zip_code']:
    if zip_code not in printed_zip_codes:
        print("Zip Code:", zip_code)
        printed_zip_codes.add(zip_code)

Zip Code: 30000


In [ ]:
# Zip Code 30000 belongs to Murcia which is also found back in the Sociodemographic dataset
customer_potential.loc[(customer_potential['city'].isna()) & (customer_potential['zip_code'] == 30000), 'city'] = 'Murcia'
print(customer_potential.loc[(customer_potential['city'] == 'Murcia') & (customer_potential['zip_code'] == 30000), ['city', 'zip_code']])

         city  zip_code
35     Murcia     30000
468    Murcia     30000
511    Murcia     30000
741    Murcia     30000
904    Murcia     30000
...       ...       ...
37793  Murcia     30000
37855  Murcia     30000
37874  Murcia     30000
37885  Murcia     30000
38518  Murcia     30000

[181 rows x 2 columns]


## Store size in same format (m2)

In [ ]:
#store size in in squared ft, we change it m2 (number/10.764)
sqft_to_m2 = 0.0929
customer_potential['store_size'] = customer_potential['store_size'] * sqft_to_m2

## Opening hours

In [ ]:
customer_potential['opening_hours'].unique()


array(['11:00 AM - 11:00 PM', '9:00 AM - 9:00 PM', '6:00 AM - 11:00 PM',
       '1:00 PM - 1:00 AM', '11:00 PM - 6:00 AM', '10:00 AM - 10:30 PM',
       '12:00 AM - 12:00 AM', '1:00 PM - 2:00 AM'], dtype=object)

In [ ]:
#Creating two new columns, for opening and closing seperate
customer_potential[['opening_hour', 'closing_hour']] = customer_potential['opening_hours'].str.split(' - ', expand=True)


In [ ]:
#Change the format type
customer_potential['opening_hour'] = pd.to_datetime(customer_potential['opening_hour'], format='%I:%M %p').dt.strftime('%H:%M')
customer_potential['closing_hour'] = pd.to_datetime(customer_potential['closing_hour'], format='%I:%M %p').dt.strftime('%H:%M')

In [ ]:
customer_potential['opening_hour'].unique()


array(['11:00', '09:00', '06:00', '13:00', '23:00', '10:00', '00:00'],
      dtype=object)

In [ ]:
customer_potential['closing_hour'].unique()


array(['23:00', '21:00', '01:00', '06:00', '22:30', '00:00', '02:00'],
      dtype=object)

In [ ]:
customer_potential['opening_hour'] = pd.to_datetime(customer_potential['opening_hour'], format='%H:%M').dt.time
customer_potential['closing_hour'] = pd.to_datetime(customer_potential['closing_hour'], format='%H:%M').dt.time


In [ ]:
customer_potential['opening_hour'] = pd.to_datetime(customer_potential['opening_hour'], format='%H:%M:%S').dt.hour
customer_potential['closing_hour'] = pd.to_datetime(customer_potential['closing_hour'], format='%H:%M:%S').dt.hour

In [ ]:
#For our model we use the 'time_period'
bins = [-1, 6, 12, 18, 24]
labels = ['Late Night', 'Morning', 'Afternoon', 'Evening']

# Bin the opening and closing hours
customer_potential['opening_bin'] = pd.cut(customer_potential['opening_hour'], bins=bins, labels=labels, right=False)
customer_potential['closing_bin'] = pd.cut(customer_potential['closing_hour'], bins=bins, labels=labels, right=False)

# Combine the binned opening and closing hours
customer_potential['time_period'] = customer_potential['opening_bin'].astype(str) + ' to ' + customer_potential['closing_bin'].astype(str)

customer_potential[['opening_hour', 'opening_bin', 'closing_hour', 'closing_bin', 'time_period']].head(10)


,opening_hour,opening_bin,closing_hour,closing_bin,time_period
0,11,Morning,23,Evening,Morning to Evening
1,11,Morning,23,Evening,Morning to Evening
2,9,Morning,21,Evening,Morning to Evening
3,11,Morning,23,Evening,Morning to Evening
4,11,Morning,23,Evening,Morning to Evening
5,9,Morning,21,Evening,Morning to Evening
6,6,Morning,23,Evening,Morning to Evening
7,13,Afternoon,1,Late Night,Afternoon to Late Night
8,9,Morning,21,Evening,Morning to Evening
9,9,Morning,21,Evening,Morning to Evening


## Description


In [ ]:
#Getting insights on the uniqueness and count of the descriptions
print("Unique descriptions:", customer_potential['description'].nunique())
unique_values = customer_potential['description'].unique()
for value in unique_values:
    count = len(customer_potential[customer_potential['description'] == value])
    print(f"{value}: {count}")

Unique descriptions: 37
Whimsical ambiance, soft lighting, and aromatic bouquets of fresh herbs and flowers transport diners to the romance of the French countryside.: 1322
Vibrant eatery serving a diverse array of shareable Spanish dishes in a lively atmosphere, perfect for social gatherings and culinary exploration.: 1366
Sleek, boutique-inspired haven nestled in the heart of the city, blending modern luxury with vibrant neighborhood charm.: 1151
Savory aromas waft from the kitchen of Bella Vita, an authentic Italian restaurant, serving up traditional dishes with a modern twist.: 1351
Vibrant Spanish cuisine served with warmth and passion, authentic paella and tapas fusion, and a lively atmosphere reminiscent of Barcelona's boulevards.: 1392
Cozy minimalist abode with sleek lines, soft lighting, and plush amenities, offering intimate solitude in a vibrant urban setting.: 1112
Luxurious getaway resort with lavish suites, upscale dining, and thrilling gaming floors, where luck and ente

In [ ]:
#To make sense of the reviews we created a new dataset that devides the review into 3 main categories
customer_potential_description = pd.read_excel(data_path + '/customer_potential_descriptions.xlsx')
customer_potential_description.head(20)

,Description,Customer Category
0,A cozy gathering spot where friends and strang...,Bar/Pub
1,A luxurious haven nestled between the soothing...,Hotel
2,A rustic haven where craft beer aficionados ga...,Bar/Pub
3,"A rustic haven with wooden accents, dimly lit ...",Bar/Pub
4,A vibrant hub where friends gather to belt out...,Bar/Pub
5,Aromatic spices waft through the doors of this...,Restaurant
6,"Cozy haven with rustic charm, warm golden ligh...",Bar/Pub
7,"Cozy minimalist abode with sleek lines, soft l...",Hotel
8,Cozy retreat with plush amenities and warm hos...,Hotel
9,Economical lodgings offering no-frills comfort...,Hotel


In [ ]:
customer_potential_description.columns = customer_potential_description.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'\W', '_')

customer_potential = pd.merge(customer_potential, customer_potential_description, on='description', how='left')
customer_potential.head()

,rating,number_of_reviews,distance,city,store_size,opening_hours,phone_number,number_of_employees,free_wifi,parking,...,latitude,longitude,date_opening,id,opening_hour,closing_hour,opening_bin,closing_bin,time_period,customer_category
0,3.184,3,38.506364,Madrid,20.4380,11:00 AM - 11:00 PM,938887612,31,0,1,...,40.366157,-3.675629,2029-09-16,19914,11,23,Morning,Evening,Morning to Evening,Restaurant
1,2.454,10,32.895455,Alicante,37.6245,11:00 AM - 11:00 PM,918230016,31,1,0,...,38.362899,-0.512787,2029-09-08,10537,11,23,Morning,Evening,Morning to Evening,Restaurant
2,2.492,7,36.050057,Barcelona,427.6187,9:00 AM - 9:00 PM,946142438,33,1,0,...,41.469548,2.182110,2028-12-20,17477,9,21,Morning,Evening,Morning to Evening,Hotel
3,2.750,4,39.215338,Alicante,62.0572,11:00 AM - 11:00 PM,938399524,23,0,1,...,38.365815,-0.526732,2028-11-16,25714,11,23,Morning,Evening,Morning to Evening,Restaurant
4,2.487,12,35.995439,Madrid,80.2656,11:00 AM - 11:00 PM,950017962,30,1,1,...,40.402235,-3.709612,2028-07-04,23921,11,23,Morning,Evening,Morning to Evening,Restaurant


In [ ]:
customer_potential['free_wifi'].unique()


array([0, 1])

In [ ]:
customer_potential['parking'].unique()


array([1, 0])

In [ ]:
customer_potential['date_opening'] = pd.to_datetime(customer_potential['date_opening'])
customer_potential['date_opening_year'] = customer_potential['date_opening'].dt.year


In [ ]:
customer_potential.dtypes

rating                        float64
number_of_reviews               int64
distance                      float64
city                           object
store_size                    float64
opening_hours                  object
phone_number                    int64
number_of_employees             int64
free_wifi                       int64
parking                         int64
description                    object
zip_code                        int64
latitude                      float64
longitude                     float64
date_opening           datetime64[ns]
id                              int64
opening_hour                    int32
closing_hour                    int32
opening_bin                  category
closing_bin                  category
time_period                    object
customer_category              object
date_opening_year               int32
dtype: object

In [ ]:
#Created variables not needed for the model so we can drop them
customer_potential = customer_potential.drop(columns=['opening_hour','closing_hour','opening_bin','closing_bin'])

In [ ]:
retain_columns = ['id', 'distance', 'city', 'opening_hours', 'phone_number', 'number_of_employees','free_wifi', 'parking', 'description', 'zip_code','latitude','longitude','date_opening', 'store_size', 'rating', 'number_of_reviews', 'time_period','customer_category']
retained_df = customer_potential[retain_columns]

In [ ]:
#One hot encoding of categorical variables for the model
customer_potential = pd.get_dummies(customer_potential, columns=['time_period','customer_category'], drop_first=True)

In [ ]:
modeling_df = customer_potential.drop(columns=['id', 'distance', 'city', 'opening_hours', 'phone_number', 'number_of_employees','free_wifi', 'parking', 'description', 'zip_code','latitude','longitude','date_opening'])

In [ ]:
model_df = pd.read_csv(model_path + '/df.columns.csv')

In [ ]:
model_time_periods = [col for col in model_df.columns if col.startswith('time_period_') and col != 'time_period_Late Night to Afternoon']
model_time_periods

['time_period_Evening to Morning',
 'time_period_Late Night to Late Night',
 'time_period_Morning to Evening']

In [ ]:
model_customer_categories = [col for col in model_df.columns if col.startswith('customer_category_')]
model_customer_categories

['customer_category_Hotel', 'customer_category_Restaurant']

In [ ]:
for col in model_time_periods + model_customer_categories + model_customer_city:
    if col not in modeling_df.columns:
        modeling_df[col] = 0

# Applying our Linear Regression model to predict sales

In [ ]:
features = [
    'rating', 'number_of_reviews', 'date_opening_year'
] + model_time_periods + model_customer_categories

In [ ]:
print(features)

['rating', 'number_of_reviews', 'date_opening_year', 'time_period_Evening to Morning', 'time_period_Late Night to Late Night', 'time_period_Morning to Evening', 'customer_category_Hotel', 'customer_category_Restaurant']


In [ ]:
X_new = modeling_df[features]


In [ ]:
X_new = sm.add_constant(X_new)


In [ ]:
with open(model_path + '/model.pkl','rb') as file:
  model = pickle.load(file)

In [ ]:
predictions = model.predict(X_new)

In [ ]:
retained_df['predicted_sales'] = predictions

In [ ]:
output_dir = '/content/drive/Shared drives/Deloitte/Cleaned Data'

retained_df.to_csv(output_dir + '/data_with_predictions.csv', index=False)


In [ ]:
retained_df

,id,distance,city,opening_hours,phone_number,number_of_employees,free_wifi,parking,description,zip_code,latitude,longitude,date_opening,store_size,rating,number_of_reviews,time_period,customer_category,predicted_sales
0,19914,38.506364,Madrid,11:00 AM - 11:00 PM,938887612,31,0,1,"Whimsical ambiance, soft lighting, and aromati...",28000,40.366157,-3.675629,2029-09-16,20.4380,3.184,3,Morning to Evening,Restaurant,41558.398884
1,10537,32.895455,Alicante,11:00 AM - 11:00 PM,918230016,31,1,0,Vibrant eatery serving a diverse array of shar...,3000,38.362899,-0.512787,2029-09-08,37.6245,2.454,10,Morning to Evening,Restaurant,41394.368904
2,17477,36.050057,Barcelona,9:00 AM - 9:00 PM,946142438,33,1,0,"Sleek, boutique-inspired haven nestled in the ...",8000,41.469548,2.182110,2028-12-20,427.6187,2.492,7,Morning to Evening,Hotel,40959.382335
3,25714,39.215338,Alicante,11:00 AM - 11:00 PM,938399524,23,0,1,Savory aromas waft from the kitchen of Bella V...,3000,38.365815,-0.526732,2028-11-16,62.0572,2.750,4,Morning to Evening,Restaurant,40981.396868
4,23921,35.995439,Madrid,11:00 AM - 11:00 PM,950017962,30,1,1,Vibrant Spanish cuisine served with warmth and...,28000,40.402235,-3.709612,2028-07-04,80.2656,2.487,12,Morning to Evening,Restaurant,41342.400128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38566,10427,28.010804,Sevilla,10:00 AM - 10:30 PM,936170704,31,1,0,Vibrant eatery serving authentic Cantonese and...,41000,37.385244,-5.993674,2008-03-19,50.8163,5.000,122,Morning to Evening,Restaurant,47450.114449
38567,20956,36.723765,A Coruña,9:00 AM - 9:00 PM,933485900,29,0,1,Luxurious getaway haven nestled amidst lush gr...,15000,43.352722,-8.397329,2008-01-01,341.5004,5.000,329,Morning to Evening,Hotel,63319.990845
38568,19942,31.264910,Zaragoza,9:00 AM - 9:00 PM,949320299,32,1,1,Luxurious retreats wrapped in elegance and sop...,50000,41.640306,-0.835531,2007-12-08,216.5499,5.000,87,Morning to Evening,Hotel,44524.654147
38569,33810,26.856137,Madrid,9:00 AM - 9:00 PM,931646259,29,1,0,"Elegant retreat nestled amidst lush gardens, o...",28000,40.398011,-3.697531,2007-12-03,352.2768,5.000,166,Morning to Evening,Hotel,50582.945953
